In [4]:
def accuracy(TP, TN, FP, FN):
    return (TP+TN)/(TP+TN+FP+FN)


def sensitivty(TP, P):
    return TP/P


def specificity(TN, N):
    return TN/N


# random accuracy
def rand_acc(TP, TN, FP, FN, T):
    return ((TN+FP)*(TN+FN)+(FN+TP)*(FP+TP))/(T*T)


# https://www.standardwisdom.com/2011/12/29/confusion-matrix-another-single-value-metric-kappa-statistic/
def cohens_kappa(TP, TN, FP, FN, T):
    return 1-((1-accuracy(TP, TN, FP, FN)) / (1-rand_acc(TP, TN, FP, FN, T)))


In [3]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as st
import seaborn as sn

In [118]:
def main():

    #df = pd.read_csv( 'C:/Users/1evsa/Desktop/M/proj/rootanalysis/dados/Mock_dataset/MockDataset_train.csv')
    df = pd.read_csv('C:/Users/1evsa/Desktop/M/proj/rootanalysis/dados/Public_dataset/PublicDataset_train.csv')
    
    df['DerivedFlag'] = df['DerivedFlag'].fillna(0)  # retirar nan da coluna
    df['Date'] = df['Date'].str.replace('-', '')
    df = df.astype({'DerivedFlag': 'int32'})

    # data frames to save the best results from granger causality tests
    df_ssrf = pd.DataFrame()
    #df_ssrchi = pd.DataFrame()
    #df_lrtest = pd.DataFrame()
    #df_parmf = pd.DataFrame()

    groups = pd.unique(df['GroupKey'])
    #groups = groups[:1]
    for group in groups:
        try:
            gr = df[df['GroupKey'] == group]
            pks = pd.unique(gr['PrimaryKey'])
            root = df[df['PrimaryKey'] == group][['Date', 'Value']]
    
            _ssrf = {}
            #_ssrchi = {}
            #_lrtest = {}
            #_parmf = {}
    
            num_lags = 10
            pks = pks[1:]  # remove A1(root) form keys
            for key in pks:
                print(group, key)
                
                flag = df.loc[(df['PrimaryKey'] == key), 'DerivedFlag'].values[0]
                rel = df[df['PrimaryKey'] == key][['Date', 'Value']]
                VAR = pd.merge(root, rel, how='outer', on='Date')
    
                gc = st.grangercausalitytests(
                    VAR[['Value_x', 'Value_y']], maxlag=num_lags, verbose=False)
    
                ssrf = {'score': 0, 'lag': 0, 'flag': flag, 'prediction': 0}
                #ssrchi = {'score': 0, 'lag': 0, 'flag': flag, 'prediction': 0}
                #lrtest = {'score': 0, 'lag': 0, 'flag': flag, 'prediction': 0}
                #parmf = {'score': 0, 'lag': 0, 'flag': flag, 'prediction': 0}
    
                for lag, item in gc.items():
                    if item[0]['ssr_ftest'][0] > ssrf['score']:
                        ssrf['score'] = item[0]['ssr_ftest'][0]
                        ssrf['lag'] = lag
                    #if item[0]['ssr_chi2test'][0] > ssrchi['score']:
                    #    ssrchi['score'] = item[0]['ssr_chi2test'][0]
                    #    ssrchi['lag'] = lag
                    #if item[0]['lrtest'][0] > lrtest['score']:
                    #    lrtest['score'] = item[0]['lrtest'][0]
                    #    lrtest['lag'] = lag
                    #if item[0]['params_ftest'][0] > parmf['score']:
                    #    parmf['score'] = item[0]['params_ftest'][0]
                    #    parmf['lag'] = lag
    
                _ssrf[key] = ssrf
                #_ssrchi[key] = ssrchi
                #_lrtest[key] = lrtest
                #_parmf[key] = parmf

            df_ssrf = pd.concat( [df_ssrf, pd.DataFrame.from_dict(_ssrf, orient='index')])
            #df_ssrchi = pd.concat( [df_ssrchi, pd.DataFrame.from_dict(_ssrchi, orient='index')])
            #df_lrtest = pd.concat([df_lrtest, pd.DataFrame.from_dict(_lrtest, orient='index')])
            #df_parmf = pd.concat([df_parmf, pd.DataFrame.from_dict(_parmf, orient='index')])
        except Exception as inst:
            print(inst)
            print(group, key)
            print(df[df['PrimaryKey'] == key])
            
   

    tests = []
    tests += (conf_m_analysis(df_ssrf, range(1, 4), 'ssr_ftest'))
    
    #tests += (conf_m_analysis(df_ssrf, [5, 10, 15, 25, 40], 'ssr_chi2test'))
    #tests += (conf_m_analysis(df_ssrf, [5, 10, 15, 25, 40], 'lrtest'))
    #tests += (conf_m_analysis(df_ssrf, [5, 10, 15, 25, 40], 'params_ftest'))

    t = pd.DataFrame(tests)
    print(t)
    
    return t


def conf_m_analysis(df, thresholds, m):
    metrics = []
    for threshold in thresholds:
        df['prediction'] = df['prediction'] = 0

        df.loc[df['score'] > threshold, ['prediction']] = 1
        conf_m = pd.crosstab(df['flag'], df['prediction'])
        
        T = (len(df))
        P = len(df[df['flag'] == 1])
        N = T - P
        TN = conf_m[0][0]
        FP = conf_m[0][1]
        FN = conf_m[1][0]
        TP = conf_m[1][1]

        acc = accuracy(TP, TN, FP, FN)
        sen = sensitivty(TP, P)
        spe = specificity(TN, N)
        kappa = cohens_kappa(TP, TN, FP, FN, T)

        metrics.append({'test': m, 'threshold': threshold,
                        'acc':  acc, 'sen': sen, 'spe': spe, ' kappa': kappa})
    return metrics




# SHOW HEAT MAP DA MATRIZ DE CONFUSÃO
# conf_m = conf_m.apply(lambda r: r/r.sum(), axis=1)  # percentagens
# sn.heatmap(conf_m, annot=True, fmt='.2f')
# plt.show()


In [119]:
t = main()

WDI-PRT-NY.GDP.MKTP.CD WDI-PRT-SP.POP.TOTL
WDI-PRT-NY.GDP.MKTP.CD WDI-PRT-NE.IMP.GNFS.ZS
WDI-PRT-NY.GDP.MKTP.CD WDI-PRT-NE.EXP.GNFS.ZS
WDI-PRT-NY.GDP.MKTP.CD WDI-PRT-FP.CPI.TOTL.ZG
WDI-PRT-NY.GDP.MKTP.CD WDI-PRT-NY.GDP.PCAP.CD
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_21-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_35-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_33-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_11-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmv_8-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_32-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_10-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_9-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmeas_13-1-PRT
WDI-PRT-NY.GDP.MKTP.CD TEP-xmv_7-1-PRT
WDI-PRT-NY.GDP.MKTP.CD DJI-INTC-high-PRT
WDI-PRT-NY.GDP.MKTP.CD DJI-CAT-high-PRT
WDI-PRT-NY.GDP.MKTP.CD DJI-T-high-PRT
WDI-PRT-NY.GDP.MKTP.CD DJI-BAC-high-PRT
WDI-PRT-NY.GDP.MKTP.CD DJI-KRFT-high-PRT
WDI-USA-NY.GDP.MKTP.CD WDI-USA-SP.POP.TOTL
WDI-USA-NY.GDP.MKTP.CD WDI-USA-NE.IMP.GNFS.ZS
WDI-USA-NY.GDP.MKTP.CD WDI-USA-NE.EXP.GNFS.ZS
WDI-USA-NY.GDP.MKTP.CD WD

c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full 

 TEP-xmeas_8-1-USA
WDI-USA-NY.GDP.MKTP.CD TEP-xmeas_24-1-USA
WDI-USA-NY.GDP.MKTP.CD TEP-xmeas_34-1-USA
WDI-USA-NY.GDP.MKTP.CD TEP-xmeas_12-1-USA
WDI-USA-NY.GDP.MKTP.CD TEP-xmv_2-1-USA
WDI-USA-NY.GDP.MKTP.CD DJI-VZ-high-USA
WDI-USA-NY.GDP.MKTP.CD DJI-KRFT-high-USA
WDI-USA-NY.GDP.MKTP.CD DJI-KO-high-USA
WDI-USA-NY.GDP.MKTP.CD DJI-AA-high-USA
WDI-USA-NY.GDP.MKTP.CD DJI-GE-high-USA
WDI-GBR-NY.GDP.MKTP.CD WDI-GBR-SP.POP.TOTL
WDI-GBR-NY.GDP.MKTP.CD WDI-GBR-NE.IMP.GNFS.ZS
WDI-GBR-NY.GDP.MKTP.CD WDI-GBR-NE.EXP.GNFS.ZS
WDI-GBR-NY.GDP.MKTP.CD WDI-GBR-FP.CPI.TOTL.ZG
WDI-GBR-NY.GDP.MKTP.CD WDI-GBR-NY.GDP.PCAP.CD
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_6-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_28-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmv_9-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_27-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_39-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_11-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_7-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_22-1-GBR
WDI-GBR-NY.GDP.MKTP.CD TEP-xmeas_38-1-GBR
WDI-GBR-NY.GDP.MKTP

c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 7, but rank is 4
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 4
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full 

 DJI-INTC-high-GBR
WDI-DEU-NY.GDP.MKTP.CD WDI-DEU-SP.POP.TOTL
WDI-DEU-NY.GDP.MKTP.CD WDI-DEU-NE.IMP.GNFS.ZS
WDI-DEU-NY.GDP.MKTP.CD WDI-DEU-NE.EXP.GNFS.ZS
WDI-DEU-NY.GDP.MKTP.CD WDI-DEU-FP.CPI.TOTL.ZG
WDI-DEU-NY.GDP.MKTP.CD WDI-DEU-NY.GDP.PCAP.CD
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_25-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_26-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_12-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmv_1-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_20-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmv_5-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_37-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_29-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_1-1-DEU
WDI-DEU-NY.GDP.MKTP.CD TEP-xmeas_38-1-DEU
WDI-DEU-NY.GDP.MKTP.CD DJI-JPM-high-DEU
WDI-DEU-NY.GDP.MKTP.CD DJI-DIS-high-DEU
WDI-DEU-NY.GDP.MKTP.CD DJI-CVX-high-DEU
WDI-DEU-NY.GDP.MKTP.CD

c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 3
  warnings.warn('covariance of constraints does not have full '
c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full 

 DJI-KO-high-DEU
WDI-DEU-NY.GDP.MKTP.CD DJI-MSFT-high-DEU
WDI-FRA-NY.GDP.MKTP.CD WDI-FRA-SP.POP.TOTL
WDI-FRA-NY.GDP.MKTP.CD WDI-FRA-NE.IMP.GNFS.ZS
WDI-FRA-NY.GDP.MKTP.CD WDI-FRA-NE.EXP.GNFS.ZS
WDI-FRA-NY.GDP.MKTP.CD WDI-FRA-FP.CPI.TOTL.ZG
WDI-FRA-NY.GDP.MKTP.CD WDI-FRA-NY.GDP.PCAP.CD
WDI-FRA-NY.GDP.MKTP.CD TEP-xmv_11-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_11-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_16-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmv_10-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_7-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmv_2-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_35-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_29-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_24-1-FRA
WDI-FRA-NY.GDP.MKTP.CD TEP-xmeas_19-1-FRA
WDI-FRA-NY.GDP.MKTP.CD DJI-INTC-high-FRA
WDI-FRA-NY.GDP.MKTP.CD DJI-MSFT-high-FRA
WDI-FRA-NY.GDP.MKTP.CD

c:\users\1evsa\appdata\local\programs\python\python39\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '


 DJI-AXP-high-FRA
WDI-FRA-NY.GDP.MKTP.CD DJI-JPM-high-FRA
WDI-FRA-NY.GDP.MKTP.CD DJI-XOM-high-FRA
WDI-CHN-NY.GDP.MKTP.CD WDI-CHN-SP.POP.TOTL
WDI-CHN-NY.GDP.MKTP.CD WDI-CHN-NE.IMP.GNFS.ZS
WDI-CHN-NY.GDP.MKTP.CD WDI-CHN-NE.EXP.GNFS.ZS
WDI-CHN-NY.GDP.MKTP.CD WDI-CHN-FP.CPI.TOTL.ZG
x contains NaN or inf values.
WDI-CHN-NY.GDP.MKTP.CD WDI-CHN-FP.CPI.TOTL.ZG
                    GroupKey              PrimaryKey             RelationKey  \
4700  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4701  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4702  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4703  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4704  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4705  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4706  WDI-CHN-NY.GDP.MKTP.CD  WDI-CHN-FP.CPI.TOTL.ZG  WDI-CHN-NY.GDP.MKTP.CD   
4707 

In [115]:
t.sort_values(by=[' kappa'], ascending=False)

,test,threshold,acc,sen,spe,kappa
1,ssr_ftest,2,0.63,0.68,0.613333,0.229167
2,ssr_ftest,3,0.71,0.40,0.813333,0.216216
0,ssr_ftest,1,0.51,1.00,0.346667,0.209677


In [116]:
t.sort_values(by=['acc'], ascending=False)

,test,threshold,acc,sen,spe,kappa
2,ssr_ftest,3,0.71,0.40,0.813333,0.216216
1,ssr_ftest,2,0.63,0.68,0.613333,0.229167
0,ssr_ftest,1,0.51,1.00,0.346667,0.209677


In [ ]:
df

In [105]:
gr = df[df['GroupKey'] == 'WDI-PRT-NY.GDP.MKTP.CD']
gr

,GroupKey,PrimaryKey,RelationKey,RelationType,Date,Value,Dataset,InfoString,DerivedFlag
0,WDI-PRT-NY.GDP.MKTP.CD,WDI-PRT-NY.GDP.MKTP.CD,NaN,ROOT,2020-01-01,9.201604e+09,WDI,GDP (current US$),NaN
1,WDI-PRT-NY.GDP.MKTP.CD,WDI-PRT-NY.GDP.MKTP.CD,NaN,ROOT,2020-02-01,1.123912e+10,WDI,GDP (current US$),NaN
2,WDI-PRT-NY.GDP.MKTP.CD,WDI-PRT-NY.GDP.MKTP.CD,NaN,ROOT,2020-03-01,1.509056e+10,WDI,GDP (current US$),NaN
3,WDI-PRT-NY.GDP.MKTP.CD,WDI-PRT-NY.GDP.MKTP.CD,NaN,ROOT,2020-04-01,1.751239e+10,WDI,GDP (current US$),NaN
4,WDI-PRT-NY.GDP.MKTP.CD,WDI-PRT-NY.GDP.MKTP.CD,NaN,ROOT,2020-05-01,1.934761e+10,WDI,GDP (current US$),NaN
...,...,...,...,...,...,...,...,...,...
898,WDI-PRT-NY.GDP.MKTP.CD,DJI-KRFT-high-PRT,WDI-PRT-NY.GDP.MKTP.CD,RELATED,2023-03-01,3.188000e+01,DJI,"Dow Jones Index, high",0.0
899,WDI-PRT-NY.GDP.MKTP.CD,DJI-KRFT-high-PRT,WDI-PRT-NY.GDP.MKTP.CD,RELATED,2023-04-01,3.354000e+01,DJI,"Dow Jones Index, high",0.0
900,WDI-PRT-NY.GDP.MKTP.CD,DJI-KRFT-high-PRT,WDI-PRT-NY.GDP.MKTP.CD,RELATED,2023-05-01,3.367000e+01,DJI,"Dow Jones Index, high",0.0
901,WDI-PRT-NY.GDP.MKTP.CD,DJI-KRFT-high-PRT,WDI-PRT-NY.GDP.MKTP.CD,RELATED,2023-06-01,3.390000e+01,DJI,"Dow Jones Index, high",0.0


In [107]:
pd.unique(df['PrimaryKey'])

array(['WDI-PRT-NY.GDP.MKTP.CD', 'WDI-PRT-SP.POP.TOTL',
       'WDI-PRT-NE.IMP.GNFS.ZS', 'WDI-PRT-NE.EXP.GNFS.ZS',
       'WDI-PRT-FP.CPI.TOTL.ZG', 'WDI-PRT-NY.GDP.PCAP.CD',
       'TEP-xmeas_21-1-PRT', 'TEP-xmeas_35-1-PRT', 'TEP-xmeas_33-1-PRT',
       'TEP-xmeas_11-1-PRT', 'TEP-xmv_8-1-PRT', 'TEP-xmeas_32-1-PRT',
       'TEP-xmeas_10-1-PRT', 'TEP-xmeas_9-1-PRT', 'TEP-xmeas_13-1-PRT',
       'TEP-xmv_7-1-PRT', 'DJI-INTC-high-PRT', 'DJI-CAT-high-PRT',
       'DJI-T-high-PRT', 'DJI-BAC-high-PRT', 'DJI-KRFT-high-PRT',
       'WDI-USA-NY.GDP.MKTP.CD', 'WDI-USA-SP.POP.TOTL',
       'WDI-USA-NE.IMP.GNFS.ZS', 'WDI-USA-NE.EXP.GNFS.ZS',
       'WDI-USA-FP.CPI.TOTL.ZG', 'WDI-USA-NY.GDP.PCAP.CD',
       'TEP-xmeas_27-1-USA', 'TEP-xmeas_11-1-USA', 'TEP-xmeas_37-1-USA',
       'TEP-xmeas_7-1-USA', 'TEP-xmeas_15-1-USA', 'TEP-xmeas_8-1-USA',
       'TEP-xmeas_24-1-USA', 'TEP-xmeas_34-1-USA', 'TEP-xmeas_12-1-USA',
       'TEP-xmv_2-1-USA', 'DJI-VZ-high-USA', 'DJI-KRFT-high-USA',
       'DJI-KO-high-U